In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import torch
import math

from modules.util.util import importstr
from modules.util.logconf import logging
log = logging.getLogger('nb')

In [3]:
def run(app, *argv):    
    torch.cuda.empty_cache()
    
    argv = list(argv)
    argv.insert(0, '--num-workers=4')
    log.info("Running: {}({!r}).main()".format(app, argv))
    
    app_cls = importstr(*app.rsplit('.', 1))
    app_cls(argv).main()
    
    log.info("Finished: {}.{!r}.main()".format(app, argv))

In [5]:
# width_irc=(16,128,128)
# run('prepcache.CovidPrepCacheApp', '--width-irc', f'{width_irc[0]}', f'{width_irc[1]}', f'{width_irc[2]}')

2020-11-29 23:30:57,921 INFO     pid:38635 nb:006:run Running: prepcache.CovidPrepCacheApp(['--num-workers=4', '--width-irc', '16', '128', '128']).main()
2020-11-29 23:30:57,923 INFO     pid:38635 prepcache:039:main Starting CovidPrepCacheApp, Namespace(batch_size=200, num_workers=4, width_irc=['16', '128', '128'])
100%|██████████| 1/1 [01:37<00:00, 97.63s/it]
2020-11-29 23:32:35,586 INFO     pid:38635 nb:011:run Finished: prepcache.CovidPrepCacheApp.['--num-workers=4', '--width-irc', '16', '128', '128'].main()


## Basic Unet 3D

In [4]:
batch_size = 2
epochs = 50
steps_per_epoch=200
val_cadence = 5
recall_priority = 0
unet_depth=3
pad_types = ['zero', 'replicate']
windows = ['lung', 'mediastinal', 'shifted_lung']
project_name = 'covid19_seg'
width_irc = (12,192,192)

In [5]:
model_path = 'saved-models/2020-11-22_11.01.15.best.state'

In [ ]:
run_name = 'unet3d train run p2'

In [1]:
run('training.CovidSegmentationTrainingApp', f'--epochs={epochs}', f'--batch-size={batch_size}', 
    f'--steps-per-epoch={steps_per_epoch}', f'--val-cadence={val_cadence}', f'--recall-priority={recall_priority}',
    f'--depth={unet_depth}', f'--pad-type={pad_types[1]}', '--augmented', f'--model-path={model_path}',
    '--width-irc', f'{width_irc[0]}', f'{width_irc[1]}', f'{width_irc[2]}', 
    f'--project-name={project_name}', f'--run-name={run_name}')

# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

## Covid Seg Net

In [4]:
batch_size = 2
epochs = 600
steps_per_epoch=160
val_cadence = 5
depth = 3
windows = ['lung', 'mediastinal', 'shifted_lung']
project_name = 'covid19_seg'
width_irc = (16,128,128)

In [6]:
run_name = 'CSN Train Run, 1CLR, AllAug'
notes = 'First train run using Covid Seg Net with all augmentations and one cycle learning'

In [7]:
run('training_beta.CovidSegmentationTrainingApp', f'--epochs={epochs}', f'--batch-size={batch_size}', 
    f'--ct-window={windows[2]}', f'--val-cadence={val_cadence}', f'--depth={depth}', '--augmented',
    '--width-irc', f'{width_irc[0]}', f'{width_irc[1]}', f'{width_irc[2]}', f'--notes={notes}',
    f'--steps-per-epoch={steps_per_epoch}', f'--project-name={project_name}', f'--run-name={run_name}')


# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

2020-11-30 17:32:27,253 INFO     pid:97735 nb:006:run Running: training_beta.CovidSegmentationTrainingApp(['--num-workers=4', '--epochs=600', '--batch-size=2', '--ct-window=shifted_lung', '--val-cadence=5', '--depth=3', '--augmented', '--width-irc', '16', '128', '128', '--notes=First train run using Covid Seg Net with all augmentations and one cycle learning', '--steps-per-epoch=160', '--project-name=covid19_seg', '--run-name=CSN Train Run, 1CLR, AllAug']).main()
2020-11-30 17:32:29,348 INFO     pid:97735 wandb:606:_apply_source_login setting login settings: {}
wandb: Currently logged in as: peterbacalso (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


2020-11-30 17:32:30,674 INFO     pid:97735 training_beta:216:init_model Using CUDA; 1 devices.
2020-11-30 17:32:33,110 INFO     pid:97735 modules.dsets:316:__init__ TrainingCovid2dSegmentationDataset: training mode, 159 uid's
2020-11-30 17:32:33,111 INFO     pid:97735 modules.dsets:341:__init__ TrainingCovid2dSegmentationDataset: (16, 128, 128) width_irc,  160 steps_per_epoch
2020-11-30 17:32:33,112 INFO     pid:97735 modules.dsets:316:__init__ Covid2dSegmentationDataset: validation mode, 40 uid's
2020-11-30 17:32:33,115 INFO     pid:97735 training_beta:503:main Starting CovidSegmentationTrainingApp, Namespace(augment_flip=False, augment_noise=False, augment_offset=False, augment_rotate=False, augment_scale=False, augmented=True, batch_size=2, ct_window='shifted_lung', depth=3, epochs=600, model_path=None, notes='First train run using Covid Seg Net with all augmentations and one cycle learning', num_workers=4, project_name='covid19_seg', run_name='CSN Train Run, 1CLR, AllAug', steps_pe

epoch,loss/trn,dice_loss/trn,ce_loss/trn,loss/val,dice_loss/val,ce_loss/val,metrics_val/miss_rate,metrics_val/fp_to_mask_ratio,pr_val/precision,pr_val/recall,pr_val/f1_score
1,0.7425,0.8641,0.6210,0.7742,0.9825,0.5659,0.6223,40.7686,0.0092,0.3777,0.0179
5,0.6661,0.9121,0.4202,0.6672,0.9635,0.3708,0.8870,3.2576,0.0335,0.1130,0.0517
10,0.5872,0.7805,0.3940,0.5752,0.8209,0.3294,0.7725,0.3550,0.3906,0.2275,0.2876
15,0.5332,0.6878,0.3787,0.5686,0.8116,0.3256,0.7564,0.3492,0.4110,0.2436,0.3059
20,0.5126,0.6514,0.3737,0.6022,0.8802,0.3242,0.8522,0.2802,0.3453,0.1478,0.2070
25,0.5135,0.6524,0.3747,0.5776,0.8317,0.3236,0.7738,0.4368,0.3412,0.2262,0.2720
30,0.5036,0.6364,0.3709,0.5869,0.8517,0.3220,0.8161,0.2963,0.3830,0.1839,0.2485
35,0.4939,0.6136,0.3742,0.5608,0.8003,0.3212,0.7492,0.3398,0.4247,0.2508,0.3154
40,0.4881,0.6085,0.3678,0.5234,0.7253,0.3214,0.5611,0.8135,0.3504,0.4389,0.3897
45,0.4903,0.6066,0.3740,0.5952,0.8701,0.3202,0.8518,0.1464,0.5031,0.1482,0.2289


2020-11-30 17:36:20,388 INFO     pid:97735 training_beta:371:log_metrics E1 CovidSegmentationTrainingApp
2020-11-30 17:36:20,389 INFO     pid:97735 training_beta:402:log_metrics E1 trn      0.7425 loss, 0.8641 dice loss, 0.6210 ce loss, 0.0888 precision, 0.6152 recall, 0.1551 f1 score 0.3848 miss rate 6.3165 fp to label ratio
2020-11-30 17:46:52,735 INFO     pid:97735 training_beta:371:log_metrics E1 CovidSegmentationTrainingApp
2020-11-30 17:46:52,736 INFO     pid:97735 training_beta:413:log_metrics E1 val      0.7742 loss, 0.9825 dice loss, 0.5659 ce_loss, 0.0092 precision, 0.3777 recall, 0.0179 f1 score 0.6223 miss rate 40.7686 fp to label ratio
2020-11-30 17:46:55,072 INFO     pid:97735 training_beta:489:save_model Saved model params to saved-models/2020-11-30_17.32.28.480.state
2020-11-30 17:46:55,078 INFO     pid:97735 training_beta:494:save_model Saved model params to saved-models/2020-11-30_17.32.28.best.state
2020-11-30 17:46:55,089 INFO     pid:97735 training_beta:499:save_mo

loss/trn,0.3637
dice_loss/trn,0.38847
ce_loss/trn,0.33893
metrics_trn/miss_rate,0.29632
metrics_trn/fp_to_mask_ratio,0.12752
pr_trn/precision,0.84658
pr_trn/recall,0.70368
pr_trn/f1_score,0.76854
loss/val,0.51761
dice_loss/val,0.71598
ce_loss/val,0.31923


loss/trn,█▅▅▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▄▂▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁
dice_loss/trn,█▅▅▄▄▃▃▃▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▄▂▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁
ce_loss/trn,█▆▅▅▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▄▂▃▂▂▃▃▂▂▂▂▂▂▂▁▁▁
metrics_trn/miss_rate,█▇▇▆▆▅▄▅▅▄▄▄▄▄▄▄▄▄▄▄▃▃▄▅▃▄▃▃▃▃▃▃▂▃▂▂▂▁▁▁
metrics_trn/fp_to_mask_ratio,█▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▂▂▂▂▁▂▁▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
pr_trn/precision,▁▅▅▆▆▆▆▇▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇▆▇▇▇▇▇▇▇▇█▇██████
pr_trn/recall,▁▂▂▃▃▄▅▄▄▅▅▅▅▅▅▅▅▅▅▅▆▆▅▄▆▅▆▆▆▆▆▆▇▆▇▇▇███
pr_trn/f1_score,▁▃▃▄▄▅▅▅▅▆▆▅▆▆▆▆▆▆▆▆▆▆▆▅▆▆▆▇▆▆▆▇▇▇▇▇████
loss/val,█▄▄▄▄▄▄▂▃▂▂▂▂▃▂▂▃▂▃▃▄▂▂▃▂▂▁▁▃▃▂▁▂▂▄▂▂▃▂▂
dice_loss/val,█▅▆▆▅▆▆▃▄▂▃▂▃▄▃▃▃▃▄▄▅▂▃▄▃▃▁▁▅▄▂▁▃▂▆▂▃▄▂▃
ce_loss/val,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


wandb: wandb version 0.10.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


RuntimeError: No active exception to reraise